# Augmented Gower Dissimilarity Matrix

A notebook for development and testing

http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.412.4155&rep=rep1&type=pdf - noting that Gower's original paper was for a similarity matrix, whereas we are coding for a dissimilarity matrix. 

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Usual-imports" data-toc-modified-id="Usual-imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Usual imports</a></span></li><li><span><a href="#Functions-to-calculate-an-augmented-Gower-dissimilarity-matrix" data-toc-modified-id="Functions-to-calculate-an-augmented-Gower-dissimilarity-matrix-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Functions to calculate an augmented Gower dissimilarity matrix</a></span></li><li><span><a href="#Test-my-version-against-the-official-version" data-toc-modified-id="Test-my-version-against-the-official-version-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Test my version against the official version</a></span></li><li><span><a href="#Test-for-sets" data-toc-modified-id="Test-for-sets-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Test for sets</a></span></li><li><span><a href="#Further-test:-numeric,-categorical-and-sets" data-toc-modified-id="Further-test:-numeric,-categorical-and-sets-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Further test: numeric, categorical and sets</a></span></li><li><span><a href="#Test-specified-weights" data-toc-modified-id="Test-specified-weights-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Test specified weights</a></span></li><li><span><a href="#A-failure-case---non-specified-set-column-treated-as-categories" data-toc-modified-id="A-failure-case---non-specified-set-column-treated-as-categories-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>A failure case - non-specified set column treated as categories</a></span></li><li><span><a href="#Done" data-toc-modified-id="Done-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Done</a></span></li></ul></div>

## Usual imports

In [1]:
import pandas as pd
from pandas.api.types import is_numeric_dtype, is_string_dtype, is_categorical_dtype
import numpy as np
from typing import List, Dict, Optional, Union
from timeit import default_timer as timer

In [2]:
# for checking against the cannonical function
import gower 

## Functions to calculate an augmented Gower dissimilarity matrix

In addition to the standard Gower dissimilarity matrix for numeric and categorical data, this function calculates a difference between python set data. Set data is useful for comparing bundles: how many in common versus those not in common. The other augmentation is that feature columns can be weighted as more important in terms of the overall difference between the observations. 

The approach is as follows:
1. For every feature (or column) $k$ in our dataset, we calculate a dissimilatiy matrix $D_{i j k}$:
    1.    ___for numeric features___: where the column in our dataset is numeric data,
    We calculate: $$D_{i j k} = \frac{\lvert k_i - k_j \rvert}{R_k} * weight_k$$ where $k_i$ is the ith value in feature $k$, and $k_j$ is the jth value in feature $k$. $R_k$ is the range of feature $k$. If a weight for $k$ is not specified, it is assumed to be 1.

    1.    ___for categorical features___: 
        1.    We set $D_{i j k} = weight_k$ if the the two individuals (rows) at $i$, and $j$ __disagree__ in respect of feature $k$. If a weight for $k$ is not specified, it is assumed to be 1.
        1.    Conversely, we set $D_{i j k} = 0$ if the the two individuals (rows) at $i$, and $j$ in respect of feature $k$ __agree__.

    1.    ___for set features___: we use the cardinality of the intersection and union of the various sets as follows to calculate the dissimilarity matrix $$D_{i j k} = \left[ 1 - \left( \frac{sizeof(k_i \cap k_j)}{sizeof(k_i \cup k_j)} \right) \right] * weight_k$$ If a weight for feature $k$ is not specified, it is assumed to be 1.


2. We sum the $K$ dissimilarity matrices. Then we take the summed matrix and divide each matrix element by the total sum of the weights, which yields the weighted average dissimilarity across all features in our dataset.
$$D_{i j} = \frac{\sum\limits^{K}_{k=1} D_{i j k}}{\sum\limits^{K}_{k=1} weight_k}$$

In [3]:
# --- private supporting functions

def _update_dissimilarity_num(dissimilarity, col_series, d_col, weight):
    col_series = col_series.to_numpy()
    comparator = col_series[d_col]
    dissimilarity[d_col] += (
        abs(col_series - comparator) 
        / (col_series.max() - col_series.min())
    ) * weight
    return dissimilarity


# runs about at about the same speed as comparing numbers above
def _update_dissimilarity_cat(dissimilarity, col_series, d_col, weight):
    # comparing category codes is faster than comparing categories
    col_series = col_series.cat.codes.to_numpy()
    comparator = col_series[d_col]
    dissimilarity[d_col] += (col_series != comparator).astype(int) * weight
    return dissimilarity


# runs about 7 times slower than a number or a category
# Moral of this story: encode strings in DataFrames as a category dtype!
def _update_dissimilarity_str(dissimilarity, col_series, d_col, weight):
    col_series = col_series.to_numpy()
    comparator = col_series[d_col]
    dissimilarity[d_col] += (col_series != comparator).astype(int) * weight
    return dissimilarity


# sets run up to 300 times slower than numbers
# Moral: use sparingly!
def _update_dissimilarity_set(dissimilarity, col_series, d_col, weight):
    col_series = col_series.to_numpy()
    comparator = col_series[d_col]
    set_similar = np.vectorize(lambda x: 
                               len(x.intersection(comparator)) 
                                / len(x.union(comparator)) )
    dissimilarity[d_col] += (1 - set_similar(col_series)) * weight
    return dissimilarity


def _select_approach_for_col(col, set_cols, col_series):
    dist_func = None
    if col in set_cols:
        dist_func = _update_dissimilarity_set
    elif is_numeric_dtype(col_series):
        dist_func = _update_dissimilarity_num
    elif is_categorical_dtype(col_series):
        dist_func = _update_dissimilarity_cat
    elif is_string_dtype(col_series):
        dist_func = _update_dissimilarity_str
    else:
        print('Error: Unknown column dtype')
        assert(False) # we should never get here
    return dist_func


def _provide_feedback(col, dist_func, weight):
    explanation = {
        _update_dissimilarity_set: 'set',
        _update_dissimilarity_num: 'number',
        _update_dissimilarity_cat: 'category',
        _update_dissimilarity_str: 'string',
    }
    print(f'Treating column »{col}« as a {explanation[dist_func]}, '
          f'with weight={weight}', end="")
    return timer()


def _provide_duration(start):
    elapsed = timer() - start
    if elapsed > 0.1:
        print(f'; {elapsed:.2f} seconds', end="")
    print()


def _add_dissim_matrix_for_col(dissimilarity, col_series, dist_func, weight):
    for d_col in range(len(col_series)):
        dissimilarity = dist_func(dissimilarity, col_series, d_col, weight)
    return dissimilarity


def _calculate_dissimilarity(df, set_cols, weights, verbose):
    dissimilarity = np.zeros([len(df), len(df)], dtype='float32')
    weighted_count = 0.0

    for col in df.columns:
        col_series = df[col]
        weight = 1 if col not in weights else weights[col]
        weighted_count += weight
        dist_func = _select_approach_for_col(col, set_cols, col_series)
        if verbose: start = _provide_feedback(col, dist_func, weight)
        dissimilarity = _add_dissim_matrix_for_col(dissimilarity, 
                                                   col_series, 
                                                   dist_func, weight)
        if verbose: _provide_duration(start)
    
    if verbose: print('finalising dissimilarity matrix')
    result = dissimilarity / weighted_count
    if verbose: print(f'Min={result.min().min():.2f}, '
                      f'Max={result.max().max():.2f}')
    return result


# --- public facing function

FloatInt = Union[float, int]
StringInt = Union[str, int]
def gower_matrix(df: pd.DataFrame, 
                 set_cols: Optional[List[StringInt]]=None,
                 weights: Optional[Dict[StringInt, FloatInt]]=None,
                 verbose: bool=False) -> np.ndarray:
    
    """Calculate a Gower Dissimilarity Matrix for a DataFrame.
       Arguments:
           df -       the pandas DataFrame
           set_cols - an optional list of DataFrame columns that contain set data
                      (sets columns are otherwise treated as strings)
           weights -  an optional dictionary of column names and associated weights 
                      (default weight for every column is 1)
           verbose -  a Boolean for additional feedback while running
       Returns:
           An Augmented Gower Dissimilarity Matrix of size n * n, where n
           is the number of rows (or observsations) in the DataFrame. """
    
    # some sanity checks (takes all of about 2ms)
    assert(df.columns.is_unique)
    assert(df.index.is_unique)
    assert(df.notna().all().all())
    
    # set up default variables
    if set_cols is None:
        set_cols = []
    if weights is None:
        weights = {}
    
    # return results
    return _calculate_dissimilarity(df, set_cols, weights, verbose)

## Test my version against the official version

In [4]:
# https://jamesmccaffrey.wordpress.com/2020/04/21/example-of-calculating-the-gower-distance/

fake = pd.DataFrame([[22,   '1',     3,     0.39,   'TRUE',   'moderate',],
                     [33,   '3',     1,     0.34,   'TRUE',   'liberal',],
                     [52,   '1',     2,     0.51,   'FALSE',  'moderate'],
                     [46,   '6',     3,     0.63,   'TRUE',   'conservative'],],
                   columns = ['Age', 'Race', 'Height', 'Income', 'IsMale', 'Politics'],
                   index = ['one', 'two', 'three', 'four'])
fake

,Age,Race,Height,Income,IsMale,Politics
one,22,1,3,0.39,TRUE,moderate
two,33,3,1,0.34,TRUE,liberal
three,52,1,2,0.51,FALSE,moderate
four,46,6,3,0.63,TRUE,conservative


In [5]:
# Official Version
official = gower.gower_matrix(fake)
official

array([[0.        , 0.58984673, 0.48563218, 0.6045977 ],
       [0.58984673, 0.        , 0.78659004, 0.7388889 ],
       [0.48563218, 0.78659004, 0.        , 0.68563217],
       [0.6045977 , 0.7388889 , 0.68563217, 0.        ]], dtype=float32)

In [6]:
# my version
mine = gower_matrix(fake, verbose=True)
mine

Treating column »Age« as a number, with weight=1
Treating column »Race« as a string, with weight=1
Treating column »Height« as a number, with weight=1
Treating column »Income« as a number, with weight=1
Treating column »IsMale« as a string, with weight=1
Treating column »Politics« as a string, with weight=1
finalising dissimilarity matrix
Min=0.00, Max=0.79


array([[0.        , 0.5898468 , 0.48563218, 0.6045977 ],
       [0.5898468 , 0.        , 0.78659004, 0.7388889 ],
       [0.48563218, 0.78659004, 0.        , 0.6856322 ],
       [0.6045977 , 0.7388889 , 0.6856322 , 0.        ]], dtype=float32)

In [7]:
# Note: differences look like rounding errors
# the two matrices are the same to seven decimal places
official -  mine

array([[ 0.0000000e+00, -5.9604645e-08,  0.0000000e+00,  0.0000000e+00],
       [-5.9604645e-08,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -5.9604645e-08],
       [ 0.0000000e+00,  0.0000000e+00, -5.9604645e-08,  0.0000000e+00]],
      dtype=float32)

In [8]:
official == mine

array([[ True, False,  True,  True],
       [False,  True,  True,  True],
       [ True,  True,  True, False],
       [ True,  True, False,  True]])

## Test for sets

In [9]:
fake2 = pd.DataFrame([
    [{'fred'}],
    [{'john'}],
    [{'fred', 'john'}],
    [{'fred', 'john', 'bill'}]],
    columns = ['data'],
    index = ['one', 'two', 'three', 'four']
)
fake2

,data
one,{fred}
two,{john}
three,"{fred, john}"
four,"{bill, fred, john}"


In [10]:
gower_matrix(fake2, set_cols=['data'], weights={'data': 5}, verbose=True)

Treating column »data« as a set, with weight=5
finalising dissimilarity matrix
Min=0.00, Max=1.00


array([[0.       , 1.       , 0.5      , 0.6666666],
       [1.       , 0.       , 0.5      , 0.6666666],
       [0.5      , 0.5      , 0.       , 0.3333333],
       [0.6666666, 0.6666666, 0.3333333, 0.       ]], dtype=float32)

## Further test: numeric, categorical and sets

In [11]:
fake3 = fake.copy()
fake3['sets'] = fake2
fake3

,Age,Race,Height,Income,IsMale,Politics,sets
one,22,1,3,0.39,TRUE,moderate,{fred}
two,33,3,1,0.34,TRUE,liberal,{john}
three,52,1,2,0.51,FALSE,moderate,"{fred, john}"
four,46,6,3,0.63,TRUE,conservative,"{bill, fred, john}"


In [12]:
gower_matrix(fake3, set_cols=['sets'], verbose=True)

Treating column »Age« as a number, with weight=1
Treating column »Race« as a string, with weight=1
Treating column »Height« as a number, with weight=1
Treating column »Income« as a number, with weight=1
Treating column »IsMale« as a string, with weight=1
Treating column »Politics« as a string, with weight=1
Treating column »sets« as a set, with weight=1
finalising dissimilarity matrix
Min=0.00, Max=0.75


array([[0.        , 0.64844006, 0.48768473, 0.6134647 ],
       [0.64844006, 0.        , 0.74564856, 0.7285714 ],
       [0.48768473, 0.74564856, 0.        , 0.63530385],
       [0.6134647 , 0.7285714 , 0.63530385, 0.        ]], dtype=float32)

## Test specified weights

In [13]:
gower_matrix(fake3, set_cols=['sets'], weights={'Age': 2, 'Height': 3}, verbose=True)

Treating column »Age« as a number, with weight=2
Treating column »Race« as a string, with weight=1
Treating column »Height« as a number, with weight=3
Treating column »Income« as a number, with weight=1
Treating column »IsMale« as a string, with weight=1
Treating column »Politics« as a string, with weight=1
Treating column »sets« as a set, with weight=1
finalising dissimilarity matrix
Min=0.00, Max=0.75


array([[0.        , 0.69057477, 0.54137933, 0.5094253 ],
       [0.69057477, 0.        , 0.68528736, 0.75333333],
       [0.54137933, 0.68528736, 0.        , 0.56471264],
       [0.5094253 , 0.75333333, 0.56471264, 0.        ]], dtype=float32)

## A failure case - non-specified set column treated as categories

In [14]:
gower_matrix(fake2, verbose=True)

Treating column »data« as a string, with weight=1
finalising dissimilarity matrix
Min=0.00, Max=1.00


array([[0., 1., 1., 1.],
       [1., 0., 1., 1.],
       [1., 1., 0., 1.],
       [1., 1., 1., 0.]], dtype=float32)

In [15]:
gower_matrix(fake3, verbose=True)

Treating column »Age« as a number, with weight=1
Treating column »Race« as a string, with weight=1
Treating column »Height« as a number, with weight=1
Treating column »Income« as a number, with weight=1
Treating column »IsMale« as a string, with weight=1
Treating column »Politics« as a string, with weight=1
Treating column »sets« as a string, with weight=1
finalising dissimilarity matrix
Min=0.00, Max=0.82


array([[0.        , 0.64844006, 0.5591133 , 0.66108376],
       [0.64844006, 0.        , 0.81707716, 0.77619046],
       [0.5591133 , 0.81707716, 0.        , 0.7305419 ],
       [0.66108376, 0.77619046, 0.7305419 , 0.        ]], dtype=float32)

## Done

In [16]:
'Finished'

'Finished'